In [18]:
import numpy as np
from scipy.optimize import minimize
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib.animation import FuncAnimation
import time
import pandas as pd
from numpy.linalg import norm
from itertools import count
from IPython import display
import seaborn as sns
from IPython.display import HTML
from scipy.io import loadmat
from operator_functions.pointer_func import pointer_vector
from operator_functions.AB_operator import A_operator,B_operator,uBC_Laplace,b_vector_creator
from operator_functions.Conjugate_gradient import CG_solver_transient,CG_solver,CG_solver_transient_Matrix
from operator_functions.bound_append import bound_appender
from operator_functions.helper_util import Func_GaussElimination
import time
from joblib import Parallel, delayed
from scipy.optimize import root
from numpy import cosh, zeros_like, mgrid, zeros

In [19]:


# parameters
nx, ny = 75, 75
hx, hy = 1./(nx-1), 1./(ny-1)

P_left, P_right = 0, 0
P_top, P_bottom = 1, 0

def residual(P):
   d2x = zeros_like(P)
   d2y = zeros_like(P)

   d2x[1:-1] = (P[2:]   - 2*P[1:-1] + P[:-2]) / hx/hx
   d2x[0]    = (P[1]    - 2*P[0]    + P_left)/hx/hx
   d2x[-1]   = (P_right - 2*P[-1]   + P[-2])/hx/hx

   d2y[:,1:-1] = (P[:,2:] - 2*P[:,1:-1] + P[:,:-2])/hy/hy
   d2y[:,0]    = (P[:,1]  - 2*P[:,0]    + P_bottom)/hy/hy
   d2y[:,-1]   = (P_top   - 2*P[:,-1]   + P[:,-2])/hy/hy

   return d2x + d2y + 5*cosh(P).mean()**2


# visualize
import matplotlib.pyplot as plt
x, y = mgrid[0:1:(nx*1j), 0:1:(ny*1j)]
plt.pcolormesh(x, y, sol.x, shading='gouraud')
plt.colorbar()
plt.show()

NameError: name 'sol' is not defined

In [20]:
##diffusivity
alpha  = 0.005 ;
## Delta t
dt = 0.02 ;
## grid
Nx=19
Ny=19
xtotal  = np.linspace(0,1,Nx+2)
x=xtotal[1:Nx+1]
ytotal  = np.linspace(0,1,Ny+2)
y=ytotal[1:Ny+1]
dx = x[1] - x[0] 
dy=y[1] - y[0] 
xv, yv = np.meshgrid(x, y, indexing='ij')
X,Y=np.meshgrid(xtotal, ytotal, indexing='ij')
kappa=0.02*np.exp(-((((xv-0.7)**2)/0.09)+(((yv-0.6)**2)/0.25)))
b=kappa/(alpha)
Q=np.zeros((Nx*Ny,1))
BCB=(-0.3)*np.ones(Nx)
BCL=0.5-0.5*np.cos(2*np.pi*y)
BCT=0.5+0.5*np.sin(4*np.pi*x-0.5*np.pi)
BCR=np.zeros(Ny)
cg_iter=10
b_residual_list=[]
cg_iter_list=[10,50,100,200,250]
#for cg_iter in cg_iter_list:
#    Steady_ss,Solution_Matrix,b_residual=CG_solver(Q,b,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR,cg_iter)
#    b_residual_list.append(b_residual)
cg_iter=100
Steady_ss,Solution_Matrix,b_residual=CG_solver(Q,b,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR,cg_iter)
N=Nx*Ny
matrix_A=[]
matrix_B=[]

BCB=(-0.3)*np.ones(Nx)
BCL=0.5-0.5*np.cos(2*np.pi*y)
BCT=0.5+0.5*np.sin(4*np.pi*x-0.5*np.pi)
BCR=np.zeros(Ny)
b_vector=b_vector_creator(kappa,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR,dt,alpha)

In [ ]:
Q_int=Q

In [97]:
def func_new(x_new,Q):
    s=B_operator(x_new,Nx,Ny,dx,dy,dt,alpha)-A_operator(Q,Nx,Ny,dx,dy,dt,alpha)-b_vector
    return s[:,0]

In [111]:
def timer_iteration(dt):
    kappa_base=0.02*np.exp(-((((xv-0.7)**2)/0.09)+(((yv-0.6)**2)/0.25)))
    kappa=dt*kappa_base
    Q=np.zeros((Nx*Ny,1))
    norm_list=[]
    b_vector=b_vector_creator(kappa,Nx,Ny,dx,dy,BCB,BCL,BCT,BCR,dt,alpha)
    b=A_operator(Q,Nx,Ny,dx,dy,dt,alpha)+b_vector
    for it in range(100):
        Q=root(func_new, np.zeros((Nx*Ny,1)), method='df-sane',args=(Q)).x
        norm_l2 = (norm(Q-Steady_ss))/(norm(Steady_ss))
        if it%10==0:
            print(it*dt,norm_l2)
        norm_list.append(norm_l2)
        if norm_l2<1e-4:
            break
        if it>3:
            if (norm_list[-1]-norm_list[-2])>0:
                break
    return Q,norm_list

In [112]:
Q,norm_list=timer_iteration(1)

0 0.9890542964092446
10 0.8869355150914063
20 0.7936964640195268
30 0.7078029016475909
40 0.6299433515600601
50 0.5618997170946339
60 0.5063739903525999
70 0.4667440243730494
80 0.44627637752577387


In [113]:
Q

array([[0.07434964],
       [0.10876884],
       [0.17639097],
       [0.26597   ],
       [0.36685927],
       [0.46838643],
       [0.56029807],
       [0.63350453],
       [0.68084689],
       [0.69774674],
       [0.68265711],
       [0.63728017],
       [0.56656198],
       [0.47849235],
       [0.38367208],
       [0.29439176],
       [0.22258259],
       [0.17547704],
       [0.14667177],
       [0.08687506],
       [0.10216094],
       [0.14067584],
       [0.19483584],
       [0.25719354],
       [0.3205798 ],
       [0.37835269],
       [0.42475127],
       [0.45528534],
       [0.46709706],
       [0.45925399],
       [0.43297424],
       [0.39183157],
       [0.34199116],
       [0.29238898],
       [0.25439404],
       [0.23996516],
       [0.257055  ],
       [0.30148755],
       [0.09051766],
       [0.09566709],
       [0.11687857],
       [0.1491046 ],
       [0.18727157],
       [0.22664224],
       [0.26298998],
       [0.2927428 ],
       [0.31314642],
       [0.322

In [ ]:
norm_list_total=[]
iter_list_total=[]
t = time.time()
results = Parallel(n_jobs=2)(delayed(timer_iteration)(dt) for dt in [0.1,1,5,10])
print(time.time()-t)